In [1]:
import pandas as pd
import re 
import nltk as nltk
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import spacy
import time
import math
import string
import gensim
import pyLDAvis
import os
import requests
import pprint
import seaborn as sns 
import matplotlib.pyplot as plt
#import pyLDAvis.gensim
#import pyLDAvis.gensim_models as gensimvis
import warnings
warnings.simplefilter('ignore')


from IPython.display import clear_output
from joblib import dump, load
from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from gensim import corpora, models
from gensim.models.ldamulticore import LdaMulticore

pyLDAvis.enable_notebook()
#nlp = spacy.load("en_core_web_sm")

In [2]:
import pandarallel
import multiprocessing
from pandarallel import pandarallel

num_processors = multiprocessing.cpu_count()

pandarallel.initialize(nb_workers=num_processors, use_memory_fs=False, progress_bar=True)

INFO: Pandarallel will run on 64 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


### Sentiment Analysis (Training the Yelp Custom Sentiment Model)

In [3]:
def get_gcs_data (bucket_name, folder_name, file_name, path_local):
    url = 'https://storage.googleapis.com/' + bucket_name + '/' + folder_name + '/' + file_name
    r = requests.get(url)
    open(path_local + '/' + file_name , 'wb').write(r.content)

In [4]:
path_yelp_data = '/home/jupyter/yelp'
path_yelp_model = '/home/jupyter/data/yelp/yelp_model/'

os.makedirs(path_yelp_data, exist_ok=True)
os.makedirs(path_yelp_model, exist_ok=True)

In [5]:
bucket_name = 'msca-bdp-data-open'
folder_name = 'yelp/yelp_model'
file_name = ['model.joblib', 
             'nb.joblib', 'logreg.joblib', 'svm.joblib',
             # 'nb_small.joblib', 'logreg_small.joblib', 'svm_small.joblib'
            ]
path_local = path_yelp_model

os.makedirs(path_local, exist_ok=True)

for file in file_name:
    get_gcs_data (bucket_name = bucket_name,
                 folder_name = folder_name,
                 file_name = file,
                 path_local = path_local)
    print('Downloaded: ' + file)

Downloaded: model.joblib
Downloaded: nb.joblib
Downloaded: logreg.joblib
Downloaded: svm.joblib


In [6]:
!ls -l /home/jupyter/data/yelp/yelp_model/

total 1848488
-rw-r--r-- 1 jupyter jupyter 522995808 May 19 19:41 logreg.joblib
-rw-r--r-- 1 jupyter jupyter   7661465 May 19 19:41 model.joblib
-rw-r--r-- 1 jupyter jupyter 839186606 May 19 19:41 nb.joblib
-rw-r--r-- 1 jupyter jupyter 522996092 May 19 19:42 svm.joblib


In [7]:
# Logistic Regression
%time clf_large = load(os.path.join(path_yelp_model, 'logreg.joblib'))

CPU times: user 42.6 s, sys: 1.99 s, total: 44.6 s
Wall time: 43.8 s


In [8]:
%%time

df = pd.read_parquet('new_df_clean_withtokens.parquet', engine='pyarrow')
df.shape

CPU times: user 14.9 s, sys: 2.66 s, total: 17.6 s
Wall time: 14.3 s


(170984, 9)

In [9]:
df.sample(3, random_state = 42)

,date,year,month,day,dayofweek,clean_title,clean_text,clean_title_tokens,clean_text_tokens
84444,2021-03-30,2021,3,30,1,Cambridge Quantum Computing Pioneers Quantum M...,Cambridge Quantum Computing Pioneers Quantum M...,"[cambridge, quantum, computing, pioneers, quan...","[cambridge, quantum, computing, pioneers, quan..."
35622,2022-12-08,2022,12,8,3,Aetina Launches New ASICBased Edge AI System P...,Aetina Launches New ASICBased Edge AI System P...,"[aetina, launches, new, asicbased, edge, ai, s...","[aetina, launches, new, asicbased, edge, ai, s..."
140578,2020-12-04,2020,12,4,4,Emotion AI Software Market To Observe Exponent...,Emotion AI Software Market To Observe Exponent...,"[emotion, ai, software, market, observe, expon...","[emotion, ai, software, market, observe, expon..."


In [10]:
X_test = df['clean_text']

In [11]:
%%time

y_pred = clf_large.predict(X_test)

CPU times: user 4min 16s, sys: 556 ms, total: 4min 17s
Wall time: 4min 17s


In [13]:
df['sentiment'] = y_pred

In [15]:
df[['clean_text', 'sentiment']].sample(10, random_state = 42)

,clean_text,sentiment
84444,Cambridge Quantum Computing Pioneers Quantum M...,1
35622,Aetina Launches New ASICBased Edge AI System P...,1
140578,Emotion AI Software Market To Observe Exponent...,0
166864,Dystopia Prime: Amazon AI van cameras spark su...,0
168822,Researcher uses AI to make texts that are thou...,0
90038,Lunit Meets MDR CE Requirements for 3D Breast ...,0
72529,US launches artificial intelligence military ...,0
179464,ChatGPTdriven smart home voice assistant comin...,1
80008,Cybersecurity Services Market Tackles HyperCo...,0
119820,Supermicro Accelerates Delivery of HPC Cluster...,0


In [17]:
# 0 is negative, 1 is positive 
df['sentiment'].value_counts()

0    101981
1     69003
Name: sentiment, dtype: int64

In [18]:
df.to_parquet('df_sentiment.parquet')